## 1. Kütüphaneleri İmport Et

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Görüntüleme ayarları
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

print("✓ Kütüphaneler başarıyla yüklendi")

## 2. Veri Setlerini Yükle

In [ ]:
# Veri setlerini yükle
coffee_df = pd.read_csv('./data/synthetic_coffee_health_10000.csv')
caffeine_df = pd.read_csv('./data/caffeine_intake_tracker.csv')
sleep_df = pd.read_csv('./data/Sleep_Efficiency.csv')

print("✓ Veri setleri yüklendi")
print(f"\nCoffee Dataset: {coffee_df.shape[0]} satır, {coffee_df.shape[1]} sütun")
print(f"Caffeine Dataset: {caffeine_df.shape[0]} satır, {caffeine_df.shape[1]} sütun")
print(f"Sleep Dataset: {sleep_df.shape[0]} satır, {sleep_df.shape[1]} sütun")

## 3. Her Veri Setinin Yapısını İnceledim

In [ ]:
print("="*80)
print("COFFEE DATASET (Global Coffee Health)")
print("="*80)
print(f"\nKolon İsimleri:")
print(coffee_df.columns.tolist())
print(f"\nVeri Tipleri:")
print(coffee_df.dtypes)
print(f"\nİlk 5 Satır:")
print(coffee_df.head())
print(f"\nTemel İstatistikler:")
print(coffee_df.describe())
print(f"\nExcik Değerler:")
print(coffee_df.isnull().sum())

In [ ]:
print("="*80)
print("CAFFEINE DATASET")
print("="*80)
print(f"\nKolon İsimleri:")
print(caffeine_df.columns.tolist())
print(f"\nVeri Tipleri:")
print(caffeine_df.dtypes)
print(f"\nİlk 5 Satır:")
print(caffeine_df.head())
print(f"\nTemel İstatistikler:")
print(caffeine_df.describe())
print(f"\nExcik Değerler:")
print(caffeine_df.isnull().sum())

In [ ]:
print("="*80)
print("SLEEP DATASET")
print("="*80)
print(f"\nKolon İsimleri:")
print(sleep_df.columns.tolist())
print(f"\nVeri Tipleri:")
print(sleep_df.dtypes)
print(f"\nİlk 5 Satır:")
print(sleep_df.head())
print(f"\nTemel İstatistikler:")
print(sleep_df.describe())
print(f"\nExcik Değerler:")
print(sleep_df.isnull().sum())

## 4. Benzer ve Yinelenen Kolonları Tespit Et

In [ ]:
from difflib import SequenceMatcher

def find_similar_columns(df1, df2, threshold=0.6):
    """
    İki dataframe arasında benzer kolon isimlerini bulur
    """
    similar = []
    for col1 in df1.columns:
        for col2 in df2.columns:
            similarity = SequenceMatcher(None, col1.lower(), col2.lower()).ratio()
            if similarity >= threshold:
                similar.append({
                    'Dataset1': col1,
                    'Dataset2': col2,
                    'Similarity': round(similarity, 2)
                })
    return similar

# Benzer kolonları bul
coffee_caffeine_similar = find_similar_columns(coffee_df, caffeine_df)
coffee_sleep_similar = find_similar_columns(coffee_df, sleep_df)
caffeine_sleep_similar = find_similar_columns(caffeine_df, sleep_df)

print("="*80)
print("COFFEE vs CAFFEINE - Benzer Kolonlar")
print("="*80)
if coffee_caffeine_similar:
    for match in coffee_caffeine_similar:
        print(f"{match['Dataset1']} <-> {match['Dataset2']} (Benzerlik: {match['Similarity']})")
else:
    print("Benzer kolon bulunamadı")

print("\n" + "="*80)
print("COFFEE vs SLEEP - Benzer Kolonlar")
print("="*80)
if coffee_sleep_similar:
    for match in coffee_sleep_similar:
        print(f"{match['Dataset1']} <-> {match['Dataset2']} (Benzerlik: {match['Similarity']})")
else:
    print("Benzer kolon bulunamadı")

print("\n" + "="*80)
print("CAFFEINE vs SLEEP - Benzer Kolonlar")
print("="*80)
if caffeine_sleep_similar:
    for match in caffeine_sleep_similar:
        print(f"{match['Dataset1']} <-> {match['Dataset2']} (Benzerlik: {match['Similarity']})")
else:
    print("Benzer kolon bulunamadı")

## 5. Eksik Değer Analizi ve Temizlik

In [ ]:
def analyze_missing_values(df, name):
    """
    Eksik değerleri analiz et ve görselleştir
    """
    print(f"\n{name} - Eksik Değer Analizi:")
    print("-" * 60)
    
    missing = df.isnull().sum()
    missing_percent = (missing / len(df)) * 100
    
    missing_df = pd.DataFrame({
        'Kolon': df.columns,
        'Eksik Sayı': missing.values,
        'Eksik %': missing_percent.values
    })
    
    missing_df = missing_df[missing_df['Eksik Sayı'] > 0].sort_values('Eksik %', ascending=False)
    
    if len(missing_df) > 0:
        print(missing_df.to_string(index=False))
    else:
        print("✓ Eksik değer yok!")
    
    return missing_df

coffee_missing = analyze_missing_values(coffee_df, "COFFEE DATASET")
caffeine_missing = analyze_missing_values(caffeine_df, "CAFFEINE DATASET")
sleep_missing = analyze_missing_values(sleep_df, "SLEEP DATASET")

In [ ]:
# Eksik değerleri temizle
print("Eksik Değer Temizliği:")
print("-" * 60)

# Satırları temizle (eksik değerleri sil)
coffee_df_clean = coffee_df.dropna()
caffeine_df_clean = caffeine_df.dropna()
sleep_df_clean = sleep_df.dropna()

print(f"Coffee: {coffee_df.shape[0]} -> {coffee_df_clean.shape[0]} satır")
print(f"Caffeine: {caffeine_df.shape[0]} -> {caffeine_df_clean.shape[0]} satır")
print(f"Sleep: {sleep_df.shape[0]} -> {sleep_df_clean.shape[0]} satır")

print("\n✓ Temizlik tamamlandı")

## 6. Yinelenen Satırları Tespit Et ve Temizle

In [ ]:
def check_duplicates(df, name):
    """
    Yinelenen satırları kontrol et
    """
    duplicates = df.duplicated().sum()
    print(f"{name}: {duplicates} yinelenen satır bulundu")
    
    if duplicates > 0:
        # İlk yinelenen satırları göster
        dup_mask = df.duplicated(keep=False)
        print(f"\nÖrnekler:")
        print(df[dup_mask].head(10))
    
    return duplicates

print("="*80)
print("YINELENEN SATIRLAR")
print("="*80)

coffee_dups = check_duplicates(coffee_df_clean, "COFFEE DATASET")
print()
caffeine_dups = check_duplicates(caffeine_df_clean, "CAFFEINE DATASET")
print()
sleep_dups = check_duplicates(sleep_df_clean, "SLEEP DATASET")

In [ ]:
# Yinelenen satırları kaldır
coffee_df_clean = coffee_df_clean.drop_duplicates()
caffeine_df_clean = caffeine_df_clean.drop_duplicates()
sleep_df_clean = sleep_df_clean.drop_duplicates()

print("✓ Yinelenen satırlar kaldırıldı")

## 7. Veri Tipleri Kontrol ve Dönüştürme

In [ ]:
def optimize_dtypes(df):
    """
    Veri tiplerini optimize et (bellek tasarrufu)
    """
    for col in df.columns:
        if df[col].dtype == 'object':
            # String sütunları kontrol et
            unique_ratio = len(df[col].unique()) / len(df[col])
            if unique_ratio < 0.05:  # Eğer benzersiz değer %5'ten azsa category kullan
                df[col] = df[col].astype('category')
        elif df[col].dtype in ['int64', 'int32']:
            # Integer sütunları optimize et
            if df[col].min() >= 0:
                if df[col].max() < 256:
                    df[col] = df[col].astype('uint8')
                elif df[col].max() < 65536:
                    df[col] = df[col].astype('uint16')
            else:
                if df[col].max() < 128 and df[col].min() > -129:
                    df[col] = df[col].astype('int8')
        elif df[col].dtype == 'float64':
            # Float sütunlarını optimize et
            df[col] = df[col].astype('float32')
    
    return df

print("Veri Tipleri Optimizasyonu:")
print("-" * 60)

print("\nCoffee Dataset Önce:")
print(coffee_df_clean.dtypes)
coffee_df_clean = optimize_dtypes(coffee_df_clean)
print("\nCoffee Dataset Sonra:")
print(coffee_df_clean.dtypes)

print("\n" + "-" * 60)
print("\nCaffeine Dataset Önce:")
print(caffeine_df_clean.dtypes)
caffeine_df_clean = optimize_dtypes(caffeine_df_clean)
print("\nCaffeine Dataset Sonra:")
print(caffeine_df_clean.dtypes)

print("\n" + "-" * 60)
print("\nSleep Dataset Önce:")
print(sleep_df_clean.dtypes)
sleep_df_clean = optimize_dtypes(sleep_df_clean)
print("\nSleep Dataset Sonra:")
print(sleep_df_clean.dtypes)

## 8. İstatistiksel Aykırı Değer Analizi (Outliers)

In [ ]:
def detect_outliers(df, name, threshold=1.5):
    """
    IQR yöntemiyle aykırı değerleri tespit et
    """
    print(f"\n{name} - Aykırı Değer Analizi (IQR):")
    print("-" * 60)
    
    outliers_info = []
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    
    for col in numeric_cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        
        lower_bound = Q1 - threshold * IQR
        upper_bound = Q3 + threshold * IQR
        
        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
        
        if len(outliers) > 0:
            outliers_info.append({
                'Kolon': col,
                'Aykırı Değer Sayısı': len(outliers),
                'Yüzde': round(len(outliers) / len(df) * 100, 2),
                'Min': outliers[col].min(),
                'Max': outliers[col].max()
            })
    
    if outliers_info:
        outliers_df = pd.DataFrame(outliers_info)
        print(outliers_df.to_string(index=False))
    else:
        print("✓ Aykırı değer bulunamadı")
    
    return outliers_info

coffee_outliers = detect_outliers(coffee_df_clean, "COFFEE DATASET")
caffeine_outliers = detect_outliers(caffeine_df_clean, "CAFFEINE DATASET")
sleep_outliers = detect_outliers(sleep_df_clean, "SLEEP DATASET")

## 9. Data Manipülasyon - Yeni Özellikler Oluştur

In [ ]:
# Sayısal sütunları belirle ve normalleştir
from sklearn.preprocessing import StandardScaler, MinMaxScaler

print("Data Manipülasyon - Normalizasyon:")
print("-" * 60)

# Coffee dataset normalizasyonu
numeric_cols_coffee = coffee_df_clean.select_dtypes(include=[np.number]).columns
if len(numeric_cols_coffee) > 0:
    scaler = StandardScaler()
    coffee_df_clean[numeric_cols_coffee] = scaler.fit_transform(coffee_df_clean[numeric_cols_coffee])
    print("✓ Coffee dataset normalize edildi")

# Caffeine dataset normalizasyonu
numeric_cols_caffeine = caffeine_df_clean.select_dtypes(include=[np.number]).columns
if len(numeric_cols_caffeine) > 0:
    scaler = StandardScaler()
    caffeine_df_clean[numeric_cols_caffeine] = scaler.fit_transform(caffeine_df_clean[numeric_cols_caffeine])
    print("✓ Caffeine dataset normalize edildi")

# Sleep dataset normalizasyonu
numeric_cols_sleep = sleep_df_clean.select_dtypes(include=[np.number]).columns
if len(numeric_cols_sleep) > 0:
    scaler = StandardScaler()
    sleep_df_clean[numeric_cols_sleep] = scaler.fit_transform(sleep_df_clean[numeric_cols_sleep])
    print("✓ Sleep dataset normalize edildi")

print("\n✓ Tüm sayısal veriler Z-score normalizasyonu ile standardize edildi")

## 10. Temizlenmiş Veri Setlerini Kaydet

In [ ]:
# Temizlenmiş veri setlerini kaydet
coffee_df_clean.to_csv('./data/coffee_cleaned.csv', index=False)
caffeine_df_clean.to_csv('./data/caffeine_cleaned.csv', index=False)
sleep_df_clean.to_csv('./data/sleep_cleaned.csv', index=False)

print("✓ Temizlenmiş veri setleri kaydedildi:")
print("  - data/coffee_cleaned.csv")
print("  - data/caffeine_cleaned.csv")
print("  - data/sleep_cleaned.csv")

## 11. Data Verification - Son Kontroller

In [ ]:
print("="*80)
print("FINAL DATA VERIFICATION")
print("="*80)

def final_verification(df, name):
    print(f"\n{name}:")
    print("-" * 60)
    print(f"Satır Sayısı: {len(df)}")
    print(f"Sütun Sayısı: {len(df.columns)}")
    print(f"Eksik Değer: {df.isnull().sum().sum()}")
    print(f"Yinelenen Satır: {df.duplicated().sum()}")
    print(f"Bellek Kullanımı: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    print(f"Veri Tipleri: {df.dtypes.value_counts().to_dict()}")
    print("✓ Verifikasyon Başarılı")

final_verification(coffee_df_clean, "COFFEE CLEANED")
final_verification(caffeine_df_clean, "CAFFEINE CLEANED")
final_verification(sleep_df_clean, "SLEEP CLEANED")

print("\n" + "="*80)
print("✓ TÜM İŞLEMLER TAMAMLANDI")
print("="*80)